In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as spa
import scipy.sparse.linalg as spalin
from scipy.optimize import fsolve
from tqdm import trange
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import numpy as np
from typing import Dict, Any
import os
import sys
sys.path.append("../dyn/")
sys.path.append("../")
sys.path.append("../phase/")
sys.path.append("../artfigs_NC/")
from dyn_ultis import *
from spatial_ultis import *
from artfigs_NC_params import *
from artfigs_NC_ultis import *

In [6]:

def analyze_parameters(param_func):
    """分析参数生成函数，返回格式化后的参数字典"""
    # 生成不同trial组合的参数实例
    p0_0 = param_func(0, 0)
    p1_0 = param_func(1, 0)
    p0_1 = param_func(0, 1)
    p_max_t1 = param_func(20, 0)
    p_max_t2 = param_func(0, 20)

    # 获取所有参数属性
    attrs = [attr for attr in dir(p0_0) if not attr.startswith('__') and not callable(getattr(p0_0, attr))]

    # 识别可变参数
    variable_params = {}
    for attr in attrs:
        # 检查trial1变化
        if getattr(p0_0, attr) != getattr(p1_0, attr):
            min_val = getattr(param_func(0, 0), attr)
            max_val = getattr(param_func(20, 0), attr)
            variable_params[attr] = (min_val, max_val)
        
        # 检查trial2变化
        elif getattr(p0_0, attr) != getattr(p0_1, attr):
            min_val = getattr(param_func(0, 0), attr)
            max_val = getattr(param_func(0, 20), attr)
            variable_params[attr] = (min_val, max_val)

    # 格式化参数值
    formatted_params = {}
    for attr in attrs:
        if attr in variable_params:
            min_v, max_v = variable_params[attr]
            formatted_params[attr] = f"{min_v:.1f}-{max_v:.1f}"
        else:
            value = getattr(p0_0, attr)
            if isinstance(value, float):
                formatted_value = f"{value:.2f}" if not value.is_integer() else f"{int(value)}"
            else:
                formatted_value = str(value)
            formatted_params[attr] = formatted_value

    return formatted_params

def generate_latex_table(
    parameters_list: List[Dict[str, Any]],
    column_headers: List[str],
    param_labels: Dict[str, str] = None
) -> str:
    """生成符合NeurIPS格式的LaTeX表格，参数为行，相图为列
    
    Args:
        parameters_list: 包含多个参数字典的列表，每个字典对应一个相图
        column_headers: 列标题列表（每个相图的名称）
        param_labels: 参数名称到LaTeX标签的映射字典
        
    Returns:
        LaTeX表格代码（需配合booktabs包使用）
    """
    # 参数显示顺序
    column_order = [
        'N_E', 'N_I',
        'N_EE', 'N_IE', 'N_EI', 'N_II',
        'd_EE', 'd_IE', 'd_EI', 'd_II',
        'g_bar_EE', 'g_bar_IE', 'g_bar_EI', 'g_bar_II',
        'g_EE', 'g_EI', 'g_IE', 'g_II'
    ]

    # 处理自定义标签
    default_labels = {col: col.replace("_", r"\_") for col in column_order}
    if param_labels:
        default_labels.update(param_labels)

    # 创建表格结构
    latex_code = [
        r"\begin{tabular}{l" + "c" * len(parameters_list) + "}",
        r"\toprule"
    ]

    # 添加表头
    headers = [r"\textbf{Parameter}"] + [rf"\textbf{{{h}}}" for h in column_headers]
    latex_code.append(" & ".join(headers) + r" \\ \midrule")

    # 添加数据行
    for param in column_order:
        label = default_labels[param]
        row = [label]
        for params in parameters_list:
            value = str(params.get(param, ""))  # 获取参数值并转为字符串
            row.append(value)
        latex_code.append(" & ".join(row) + r" \\")

    # 结束表格
    latex_code.extend([
        r"\bottomrule",
        r"\end{tabular}"
    ])

    return "\n".join(latex_code)


In [ ]:
custom_labels = {
    "g_EE": r"$\sigma_{EE}$",
    "g_EI": r"$\sigma_{EI}$",
    "g_IE": r"$\sigma_{IE}$",
    "g_II": r"$\sigma_{II}$",
    "g_bar_EE": r"$\bar{g}_{EE}$",
    "g_bar_IE": r"$\bar{g}_{IE}$",
    "g_bar_EI": r"$\bar{g}_{EI}$",
    "g_bar_II": r"$\bar{g}_{II}$",
    "d_EE": r"$d_{EE}$",
    "d_IE": r"$d_{IE}$",
    "d_EI": r"$d_{EI}$",
    "d_II": r"$d_{II}$",
    "N_EE": r"$k^{out}_{EE}$",
    "N_IE": r"$k^{out}_{IE}$",
    "N_EI": r"$k^{out}_{EI}$",
    "N_II": r"$k^{out}_{II}$",
    "N_E": r"$N_E$",
    "N_I": r"$N_I$",
}

generate_func_list = [generate_params_phase_alpha_d_II_L_chaos, generate_params_phase_g_d_II_L_chaos,\
                      generate_params_phase_wospa_g_bar_IE_g_bar_EE_L, generate_params_phase_wspa_g_bar_IE_g_bar_EE_L,\
                        generate_params_phase_d_II_g_bar_II_S, generate_params_phase_d_II_g_bar_II_thres_L]
params_list = [analyze_parameters(func) for func in generate_func_list]
columns_list = ["Fig. 5(a)", "Fig. 5(b)", "Fig. 5(c)", "Fig. 5(d)", "Fig. 5(e)", "Fig. 5(f)"]
# 生成LaTeX代码
print(generate_latex_table(params_list, columns_list, custom_labels))

\begin{tabular}{lcccccc}
\toprule
\textbf{Parameter} & \textbf{Fig. 5(a)} & \textbf{Fig. 5(b)} & \textbf{Fig. 5(c)} & \textbf{Fig. 5(d)} & \textbf{Fig. 5(e)} & \textbf{Fig. 5(f)} \\ \midrule
$N_E$ & 40000 & 40000 & 40000 & 40000 & 10000 & 40000 \\
$N_I$ & 10000 & 10000 & 10000 & 10000 & 2500 & 10000 \\
$k^{out}_{EE}$ & 62.8-439.8 & 596.90 & 1169.93 & 596.90 & 596.90 & 321.70 \\
$k^{out}_{IE}$ & 15.7-110.0 & 149.23 & 292.48 & 149.23 & 149.23 & 80.42 \\
$k^{out}_{EI}$ & 62.8-439.8 & 596.90 & 1169.93 & 1169.93 & 596.90 & 321.70 \\
$k^{out}_{II}$ & 15.7-110.0 & 149.2-859.5 & 292.48 & 292.48 & 23.9-596.9 & 29.0-205.9 \\
$d_{EE}$ & 0.05 & 0.05 & 0.07 & 0.05 & 0.10 & 0.04 \\
$d_{IE}$ & 0.05 & 0.05 & 0.07 & 0.05 & 0.10 & 0.04 \\
$d_{EI}$ & 0.05 & 0.05 & 0.07 & 0.07 & 0.10 & 0.04 \\
$d_{II}$ & 0.1-0.1 & 0.1-0.1 & 0.07 & 0.07 & 0.0-0.2 & 0.0-0.1 \\
$\bar{g}_{EE}$ & 5.50 & 5.50 & 6.3-15.3 & 6.3-9.9 & 5.50 & 0.57 \\
$\bar{g}_{IE}$ & 5 & 5 & 7.2-16.2 & 9.0-16.2 & 5 & 0.12 \\
$\bar{g}_{EI}$ & -5 & -